# pandas：处理表格和混杂数据

## pandas可以让数据清洗和分析工作变得更快更简单。pandas经常和其他工具一同使用，比如numpy和scipy，以及statsmodels和scikit-learn、matplotlib等等。

In [1]:
import pandas as pd #要使用pandas先引入，简写为pd
import numpy as np
from pandas import Series, DataFrame #从pandas引入常用的两个数据结构Series和DataFrame

##  一、pandas 数据结构——Series和DataFrame

### （一）Series

Series类似于一维数组，包括右边的数据value和左边的索引index（0到N-1），可以用series.values和series.index获取索引对象和数组表示形式

In [2]:
obj = pd.Series([4, 7, -5, 3])
obj.values
obj.index  # like range(4)

RangeIndex(start=0, stop=4, step=1)

In [3]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c']) #给每一个数据指定一个特殊的索引，比如这里的obj2['a']=-5

In [4]:
obj2['d'] = 6 #重新给索引d 赋值为6，取代了原本的4，也可以通过给索引赋值修改索引，如obj.index=['Bob','Steve','Jeff','Ryan']
obj2[['c', 'a', 'd']] #也可以通过一个索引表获取一组数据

c    3
a   -5
d    6
dtype: int64

如果用NumPy函数的话，或者对value进行计算（如过滤、标量乘法等），也都会将索引值保留下来

In [5]:
obj2[obj2 > 0]
obj2 * 2
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

也可以将Series看成一个定长的有序字典

In [6]:
'b' in obj2 #结果为 True
'e' in obj2 #结果为 False

False

可以直接通过字典来创建Series，字典的key就是index，字典的value也是Series的value。如果传入一组字典的key的话，输出的Series就会按这组key排序，新key（如California）对应的value为NaN（⾮数字，在pandas中⽤于表示缺失或NA值），而不在states中的Utah不会显示在结果中。

In [7]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)

In [8]:
pd.isnull(obj4) #pandas⽤于检测缺失数据的方法， California对应True
pd.notnull(obj4) #pandas⽤于检测缺失数据的方法， California对应False
obj4.isnull() #series中类似的检测缺失数据的方法， California对应 True

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

Series的索引标签自动对齐功能，哪怕某两个Series的索引顺序不同，但在对这两个数据做运算时，还是会分别对两个Series中的相同索引对应的value做运算，如果某一个index没有在两个Series中同时存在，那运算结果为NaN

In [9]:
obj3 #索引顺序'Ohio', 'Texas', 'Oregon', 'Utah'
obj4 #索引顺序'California', 'Ohio', 'Oregon', 'Texas'
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series对象本身及其索引都有⼀个name属性

In [10]:
obj4.name = 'population' #给Series的值命名为人口“population”
obj4.index.name = 'state' #给Series的索引命名为州“state”，那么这个Series就表示每个州对应的人口数

### （二）DataFrame

DataFrame是表格型的数据结构，包括许多列，每一列的值可以有不同的类型；DataFrame有行索引和列索引，它可以被看做由Series组成的字典（共⽤同⼀个索引）。最常⽤的⼀种建DataFrame的办法是直接传⼊⼀个由等⻓列表或NumPy数组组成的字典，结果DataFrame会⾃动加上索引（和Series⼀样），且所有列会被有序排列

In [11]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

对于特别⼤的DataFrame，head⽅法会选取前五⾏

In [12]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


如果指定了列序列，那么DataFrame的列就会按照指定的顺序进⾏排列

In [13]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


如果传⼊的列在数据中找不到，就会在结果中产⽣缺失值NaN

In [14]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])

通过类似字典标记的⽅式或属性的⽅式，可以将DataFrame的列获取为⼀个Series。frame2[column]适⽤于任何列的名，但是frame2.column只有在列名是⼀个合理的Python变量名时才适⽤

In [15]:
frame2['state']
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

⾏也可以通过loc属性获取

In [16]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的⽅式进⾏修改。例如，我们可以给那个空的"debt"列赋上⼀个标量值或⼀组值

In [17]:
frame2['debt'] = 16.5 #赋标量
frame2['debt'] = np.arange(6.) #赋一组值

将列表或数组赋值给某个列时，其⻓度必须跟DataFrame的⻓度相匹配。如果赋值的是⼀个Series，就会精确匹配DataFrame的索引，所有的空位都将被填上缺失值

In [18]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val

为不存在的列赋值会创建出⼀个新列，关键字del⽤于删除列。

In [19]:
frame2['eastern'] = frame2.state == 'Ohio'

In [20]:
del frame2['eastern'] #删除这一列

另一种创建DataFrame的形式——嵌套字典，如果嵌套字典传给DataFrame，那么外层字典的键就会是列，内层键则是⾏索引

In [21]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}
frame3 = pd.DataFrame(pop)

可以对DataFrame进⾏转置

In [22]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


### Index Objects索引对象

与python的集合不同，pandas的Index可以包含重复的标签，选择重复的标签，会显示所有的结果

In [23]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])

## 二、基本功能

### （一）重新索引

reindex将会根据新索引进⾏重排。如果某个索引值当前不存在，就引⼊缺失值

In [24]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])

对于时间序列这样的有序数据，重新索引时可能需要做⼀些插值处理。method选项即可达到此⽬的，例如，使⽤ffill可以实现前向值填充。

In [25]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

借助DataFrame，reindex可以修改（⾏）索引和列。只传递⼀个序列时，会重新索引结果的⾏

In [26]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame2 = frame.reindex(['a', 'b', 'c', 'd'])

In [27]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states) #⽤columns关键字重新索引列

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


### （二）丢弃指定轴上的项

In [28]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c') #drop⽅法返回的是⼀个在指定轴上删除了指定值的新对象
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [29]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four']) #创建DataFrame
data.drop(['Colorado', 'Ohio']) #从⾏标签（axis 0）删除值
data.drop('two', axis=1) #通过传递axis=1或axis='columns'可以删除列的值
data.drop(['two', 'four'], axis='columns') #通过传递axis=1或axis='columns'可以删除列的值

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


### （三）索引、选取和过滤

#### 1、索引

Series的索引值不只是整数

In [30]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b'] #结果为1.0
obj[1] #结果为1.0
obj[2:4] #结果是c  2.0   d   3.0
obj[['b', 'a', 'd']]
obj[[1, 3]] #结果是b  1.0   d   3.0
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

利⽤标签的切⽚运算包含末端

In [31]:
obj['b':'c']  #结果是b  1.0   c   2.0
obj['b':'c'] = 5 #⽤切⽚可以对Series的相应部分进⾏设置

⽤⼀个值或序列对DataFrame进⾏索引其实就是获取⼀个或多个列

In [32]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data['two']
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [33]:
data[:2] #通过切⽚或布尔型数组选取数据
data[data['three'] > 5] #通过切⽚或布尔型数组选取数据

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


通过布尔型DataFrame进行索引

In [34]:
data < 5
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### 2、⽤loc和iloc进⾏选取

In [35]:
data.loc['Colorado', ['two', 'three']] #通过标签选择⼀⾏和多列

two      5
three    6
Name: Colorado, dtype: int32

In [36]:
data.iloc[2, [3, 0, 1]]
data.iloc[2]
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [37]:
data.loc[:'Utah', 'two']
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


#### 3、整数索引

为了进⾏统⼀，如果轴索引含有整数，数据选取总会使⽤标签。为了更准确，应使⽤loc（标签）或iloc（整数）

In [38]:
ser = pd.Series(np.arange(3.))

In [39]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

2.0

In [40]:
ser[:1] #更准确
ser.loc[:1]
ser.iloc[:1]

0    0.0
dtype: float64

### （四）算术运算和数据对齐

pandas可以对不同索引的对象进⾏算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集

In [41]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])

In [42]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

对于DataFrame，对⻬操作会同时发⽣在⾏和列上

In [43]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [44]:
df1 + df2 #把它们相加后将会返回⼀个新的DataFrame，其索引和列为原来那两个DataFrame的并集。

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


In [45]:
df1 = pd.DataFrame({'A': [1, 2]}) #如果DataFrame对象相加，没有共⽤的列或⾏标签，结果都会是空
df2 = pd.DataFrame({'B': [3, 4]})
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


### （五）在算术方法中填充值

In [46]:
df1.add(df2, fill_value=0) #用于加法

,A,B
0,1.0,3.0
1,2.0,4.0


In [47]:
df1.rdiv(1) #用于除法

,A
0,1.0
1,0.5


In [48]:
df1.reindex(columns=df2.columns, fill_value=0) #对Series或DataFrame重新索引时，也可以指定⼀个填充值

,B
0,0
1,0


### （六）DataFrame和Series之间的运算

In [49]:
arr = np.arange(12.).reshape((3, 4))
arr - arr[0] #从arr减去arr[0]，每⼀⾏都会执⾏这个操作。这就叫做⼴播（broadcasting）

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

In [50]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0] 
frame - series #默认情况下，DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着⾏⼀直向下⼴播

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集

In [51]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


### （七）函数应用和映射

In [52]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
np.abs(frame)#用NumPy的ufuncs（元素级数组⽅法）操作pandas对象

,b,d,e
Utah,0.975554,0.143520,0.218937
Ohio,0.478794,0.489399,0.018970
Texas,0.823068,1.141792,0.202242
Oregon,0.554840,0.554456,0.243527


In [53]:
f = lambda x: x.max() - x.min()
frame.apply(f) #将函数应⽤到由各列或⾏所形成的⼀维数组上

b    1.530394
d    1.631190
e    0.445769
dtype: float64

In [54]:
frame.apply(f, axis='columns') #传递axis='columns'到apply，这个函数在每⾏执⾏

Utah      1.119074
Ohio      0.470429
Texas     1.964860
Oregon    0.798367
dtype: float64

In [55]:
format = lambda x: '%.2f' % x #想得到frame中各个浮点值的格式化字符串
frame.applymap(format)

,b,d,e
Utah,-0.98,0.14,-0.22
Ohio,-0.48,-0.49,-0.02
Texas,-0.82,1.14,0.20
Oregon,0.55,0.55,-0.24


### （八）排序和排名

In [56]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index() #使⽤sort_index⽅法，它会返回⼀个已排序的新对象

a    1
b    2
c    3
d    0
dtype: int64

In [57]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)), #DataFrame，则可以根据任意⼀个轴上的索引进⾏排序
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame.sort_index()
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [58]:
frame.sort_index(axis=1, ascending=False) #ascending=False表示降序

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [59]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values() #按值对Series进⾏排序

2   -3
3    2
0    4
1    7
dtype: int64

In [60]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values() #排序时，任何缺失值默认都会被放到Series的末尾

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

In [61]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame.sort_values(by='b') #根据⼀个或多个列中的值进⾏排序，只需要将⼀个或多个列的名字传递给sort_values的by选项即可

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


In [62]:
frame.sort_values(by=['a', 'b']) #根据多个列进⾏排序

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


In [63]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank() #rank是通过“为各组分配⼀个平均排名”的⽅式破坏平级关系的

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [64]:
obj.rank(method='first') #根据值在原数据中出现的顺序给出排名

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

### （九）带有重复标签的轴索引

In [65]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj.index.is_unique #索引的is_unique属性可以告诉你它的值是否是唯⼀的

False

如果某个索引对应多个值，则返回⼀个Series；⽽对应单个值的，则返回⼀个标量值：

In [66]:
obj['a']
obj['c']

4

## 三、汇总和计算描述统计

### （一）汇总

In [67]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])

In [68]:
df.sum() #sum⽅法将会返回⼀个含有列的和的Series

one    9.25
two   -5.80
dtype: float64

In [69]:
df.sum(axis='columns') #传⼊axis='columns'或axis=1将会按⾏进⾏求和运算

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [70]:
df.mean(axis='columns', skipna=False) #NA值会⾃动被排除，除⾮整个切⽚（这⾥指的是⾏或列）都是NA。通过skipna选项可以禁⽤该功能

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

In [71]:
df.idxmax() #间接统计（⽐如达到最⼤值的索引）

one    b
two    d
dtype: object

In [72]:
df.cumsum() #累计型

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [73]:
df.describe() #⽤于⼀次性产⽣多个汇总统计

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


### （二）相关系数与协⽅差

In [74]:
price = pd.read_pickle('examples/yahoo_price.pkl')
volume = pd.read_pickle('examples/yahoo_volume.pkl')

In [76]:
returns=price.pct_change()
returns['MSFT'].corr(returns['IBM']) #Series的corr⽅法⽤于计算两个Series中重叠的、⾮NA的、按索引对⻬的值的相关系数。与此类似，cov⽤于计算协⽅差
returns['MSFT'].cov(returns['IBM'])

8.870655479703549e-05

In [77]:
returns.MSFT.corr(returns.IBM) #更简洁的语法选择列

0.49976361144151155

In [78]:
returns.corr() #DataFrame的corr和cov⽅法将以DataFrame的形式分别返回完整的相关系数或协⽅差矩阵
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000277,0.000107,0.000078,0.000095
GOOG,0.000107,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000146,0.000089
MSFT,0.000095,0.000108,0.000089,0.000215


In [79]:
returns.corrwith(returns.IBM) #利⽤DataFrame的corrwith⽅法，你可以计算其列或⾏跟另⼀个Series或DataFrame之间的相关系数。传⼊⼀个Series将会返回⼀个相关系数值Series（针对各列进⾏计算）

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

In [80]:
returns.corrwith(volume) #传⼊⼀个DataFrame则会计算按列名配对的相关系数

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

### （三）唯⼀值、值计数以及成员资格

In [81]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [82]:
uniques = obj.unique() #unique，它可以得到Series中的唯⼀值数组，返回的唯⼀值是未排序的，如果需要的话，可以对结果再次进⾏排序（uniques.sort()）

In [83]:
obj.value_counts() #计算⼀个Series中各值出现的频率

c    3
a    3
b    2
d    1
dtype: int64

In [84]:
pd.value_counts(obj.values, sort=False) #value_counts可⽤于任何数组或序列

d    1
a    3
b    2
c    3
dtype: int64

In [85]:
mask = obj.isin(['b', 'c']) #isin⽤于判断⽮量化集合的成员资格，可⽤于过滤Series中或DataFrame列中数据的⼦集
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

In [86]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match) #是Index.get_indexer⽅法，它可以给你⼀个索引数组，从可能包含重复值的数组到另⼀个不同值的数组

array([0, 2, 1, 1, 0, 2], dtype=int64)

In [87]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
result = data.apply(pd.value_counts).fillna(0) #结果中的⾏标签是所有列的唯⼀值。这些值是每个列中这些值的相应计数。